# BAK, CHEN, TANG   Forest Fire Model

In [ ]:
%pylab inline

In [ ]:
import numpy as np
import pandas as pd
import collections as col
from matplotlib import colors
from scipy.ndimage import measurements
from scipy.ndimage import find_objects
import os

In [ ]:
#if not present, it creates the directory to save gif images
if os.path.isdir("./images_BAK_gif") == False:
    ! mkdir images_BAK_gif

## This function initializes the forest

In [ ]:
def forest_init(p,f,L):
    
    """
    This function initializes the forest with the following rules
    p = an empty space fills with a tree with probability p
    f = a tree catches fire with probability f
    L = linear dimension of the forest 
    
    The function return a 2D numpy array with shape (L,L) filled with:
    1 = tree
    2 = fire
    0 = empty
    
    the returned array is padded with zeros
    
    e.g. forest_init(0.30,0.02,50)"""
    
    if (p>1)|(p<0):
        
        return 'p must be a probability in [0,1]'

    if (f>1)|(f<0):
        
        return 'f must be a probability in [0,1]'
    
    r = rand(L-2,L-2)
    foresta = np.zeros((L-2,L-2), dtype = int)
    
    foresta[r<p] = 1 #a tree grows with probability p
    foresta[r<f] = 2 #a tree catch fire with probability f
                    
    return np.pad(foresta, pad_width=[(1,1),(1,1)], mode='constant', constant_values=0)

## This function updates the cell status

Two cells are considered neighbor according to *Von Neumann neighborhood* 

In [ ]:
def spread(i,j,p):
    """This function defines the nearest neighbour interactions and updates the [i,j] cell status;
       if a cell is burning it becomes empty and if a cell is empty is filled with a tree with probability p
    """
    
    if foresta[i][j] == 1: #if one of the neighbours of the tree is on fire the tree catches fire
       
        if (foresta[i-1][j] == 2 or foresta[i][j-1] == 2 or foresta[i+1][j] == 2 or foresta[i][j+1] == 2) :
            
            return 2            
        else:
            return 1
    
    
    if foresta[i][j] == 2: # if a cell is burning it becomes empty
        return 0
    
    if foresta[i][j] == 0: # if a cell is empty is filled with a tree with probability p 
        
        if rand() < p:
            return 1
        else:
            return 0


## This function updates the forest status

In [ ]:
def apply_spread(foresta,p):
    "This function applies the spread function to all its elements."
    
    foresta_buff = np.zeros(foresta.shape,dtype=int)
    
    for i in range(1,foresta.shape[0]-1):
        for j in range(1,foresta.shape[1]-1):
            foresta_buff[i][j] = spread(i,j,p)
    
    return foresta_buff

## Simulation

In [ ]:
L = 256
p_init = 0.40      # this is the initialization parameters
p_grow = 0.057     # after a cell is burned down, it's filled with a tree with probability p
f = 0.001

cmap1 = colors.ListedColormap(['#4D0000','#228B22','red'])
cmap2 = colors.ListedColormap(['#4D0000','#228B22'])

foresta = forest_init(p_init,f,L)
print 'number of tree on fire: ', len(foresta[foresta==2])

tmax = 200 # number of steps

if len(foresta[foresta==2])>0:  
    
    for t in range(1,tmax):
        
        foresta_buff = apply_spread(foresta,p_grow)
        foresta = foresta_buff
                
            
        if len(foresta[foresta==2])>0:
            cmap = cmap1
        else:
            cmap = cmap2
            print 'no more trees burning!'

            
        imshow(foresta_buff,cmap=cmap)
        savefig('./images_BAK_gif/forest_Fire_'+str(t)+'.png')
        show()
    
else:  
    print "there was no fire"
    

In [ ]:
from images2gif import writeGif
from PIL import Image
import os

In [ ]:
immagini = [Image.open('./images_BAK_gif/forest_Fire_'+str(t)+'.png') for t in range(1,tmax)]

In [ ]:
filename = "my_gif_p="+str(p_grow)+"_f="+str(f)+"_BAK.GIF"
writeGif(filename, immagini, duration=0.1,repeat=True)
filename